# Setup

In [ ]:
import pandas as pd
# from collections import defaultdict
# from operator import itemgetter
import numpy as np
# import re
# from fuzzywuzzy import process, fuzz

# Display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.3f}'.format)

In [ ]:
dsd = "DSD2324_overview (2).xlsx"
preferences = "Teaching Preferences Questionnaire_S1_2023-24_raw.xlsx"
contract = "ta_contract.xlsx"
bs_weights = "bs_courses_weights_FILLED.xlsx"
term = "S1"

# DEMAND - practical lessons

In [ ]:
dsd_df = pd.read_excel(dsd)

In [ ]:
dsd_df.head()

In [ ]:
dsd_df["course"] = dsd_df["COURSE CODE"].astype(str) + " || " + dsd_df["COURSE NAME"].astype(str) + " || " + dsd_df["TERM"].astype(str) + " || " + dsd_df["LANGUAGE"].astype(str)

# ADDED!
faculty_list = dsd_df[dsd_df["COURSE NAME"] != "Stata"]["FACULTY EMAIL"].unique()

if term == "S1":
    selected_terms = ["S1", "T1", "T2"]
    dsd_df = dsd_df[dsd_df['TERM'].isin(selected_terms)]
else:
    selected_terms = ["S2", "T3", "T4"]
    dsd_df = dsd_df[dsd_df['TERM'].isin(selected_terms)]

# ADDED!
teorico_practicas = dsd_df[(dsd_df["FACULTY NAME"].isna()) & (dsd_df["CYCLE"] == "BSC")]["COURSE NAME"].unique()
dsd_df = dsd_df.drop(dsd_df[(dsd_df["COURSE NAME"].isin(teorico_practicas)) & (dsd_df["FACULTY NAME"].notna())].index)

In [ ]:
agg_functions = {
    'CLASS': 'count',
    'SLOTS': np.sum
    # 'COURSE NAME': 'first'  # Add 'COURSE NAME' to the aggregation functions
}

# ADDED! "CYCLE"
output_1 = dsd_df.groupby(['COURSE NAME', 'TERM', 'COURSE CODE', 'LANGUAGE', 'CYCLE']).agg(agg_functions).reset_index()
output_1 = output_1.rename(columns={'CLASS': 'Nº CLASSES', 'SLOTS': 'Nº STUDENTS'})


output_1.head()

In [ ]:
dsd_df.TERM.value_counts(dropna=False)

In [ ]:
# ADDED!
course_demand = dsd_df[dsd_df['CYCLE'].isin(['MST', 'BSC'])]

course_demand = course_demand.groupby(['course']).agg(agg_functions).reset_index()
course_demand = course_demand.rename(columns={'CLASS': 'number_classes', 'SLOTS': 'number_students'})

# ADDED!
full_courses = dsd_df[dsd_df['CYCLE'].isin(['MST', 'BSC', 'ME'])]
full_courses = full_courses.groupby(['course', 'CYCLE', 'TERM']).agg(agg_functions).reset_index()

course_demand.head()

In [ ]:
full_courses.CYCLE.value_counts(dropna=False)

# SUPPLY - TA hours (current contract)

In [ ]:
contract = pd.read_excel(contract)

contract = contract[["TA", "CONTRACT"]]
contract["TA"] = contract["TA"].str.lower()

# Zero contracts
# ADDED! Caps "CONTRACT", remove faculty
zero_contracts = contract[contract['CONTRACT'] == 0]["TA"].unique()
contract = contract[contract['CONTRACT'] != 0]
contract = contract[~contract['TA'].isin(faculty_list)]


In [ ]:
zero_contracts

In [ ]:
# Check for duplicates
contract[contract.duplicated()].TA.count()

In [ ]:
contract_emails = contract.TA.unique()

In [ ]:
contract.head()

# SUPPLY - TA preferences

## A. Preferences

In [ ]:
# Read the Excel file with the second row as the header
preferences_df = pd.read_excel(preferences, header=1)

# Sort the DataFrame by "End Date" column in descending order
preferences_df = preferences_df.sort_values(by='End Date', ascending=False)

# Rename the column to "TA"
preferences_df.rename(columns={'Please write your E-mail @novasbe.pt': 'TA'}, inplace=True)

# Convert the values in the "TA" column to lowercase
preferences_df['TA'] = preferences_df['TA'].str.lower()

# Create a new dataframe with column names and zero-indexed column numbers
column_df = pd.DataFrame({'Column Name': preferences_df.columns,
                          'Column Number': range(len(preferences_df.columns))})

# Remove TAs with zero_contracts
preferences_df = preferences_df[~preferences_df["TA"].isin(zero_contracts)]

# ADDED! Remove TAs wicha are faculty
preferences_df = preferences_df[~preferences_df["TA"].isin(faculty_list)]

# Create a mask to identify duplicates in the "TA" column
duplicates_mask = preferences_df.duplicated(subset='TA', keep=False)
preferences_duplicates = preferences_df[duplicates_mask]
preferences_duplicates = preferences_duplicates.sort_values(by='End Date', ascending=False)

preferences_duplicates_last = preferences_duplicates.drop_duplicates(subset='TA', keep='first').copy()

# Create a mask to check if columns 31:81 or 83:347 have values
value_mask = preferences_duplicates.iloc[:, 31:81].notnull().any(axis=1) | preferences_duplicates.iloc[:, 83:347].notnull().any(axis=1)
preferences_duplicates_values = preferences_duplicates[value_mask]
preferences_duplicates_values = preferences_duplicates_values.drop_duplicates(subset='TA', keep='first').copy()

# Drop duplicates based on the "TA" column
preferences_df = preferences_df[~duplicates_mask]

# Drop duplicates based on the "Full Name" column while keeping the row with the most recent "End Date" (ex. Franziska wrong)
preferences_df = preferences_df.drop_duplicates(subset='Full Name', keep='first')

# Create a new DataFrame with columns from preferences_duplicates_last
preferences_df_final = preferences_duplicates_last.copy()

# Get the relevant columns from preferences_duplicates_values
preference_columns = preferences_duplicates_values.columns[31:81].tolist() + preferences_duplicates_values.columns[83:347].tolist()

# Update the values in preferences_df_final using values from preferences_duplicates_values for preference_columns
preferences_df_final.set_index('TA', inplace=True, drop=False)
preferences_duplicates_values.set_index('TA', inplace=True, drop=False)
preferences_df_final.loc[preferences_duplicates_values.index, preference_columns] = preferences_duplicates_values[preference_columns].values

# Concatenate the remaining columns from preferences_df to preferences_df_final
preferences_df_final = pd.concat([preferences_df_final, preferences_df])

# Sort the final DataFrame by "End Date" column in descending order
preferences_df_final.sort_values(by='End Date', ascending=False, inplace=True)

# Reset the index of the final DataFrame
preferences_df_final.reset_index(drop=True, inplace=True)

# Drop duplicates based on the "Full Name" column while keeping the row with the most recent "End Date" (ex. Franziska wrong )
preferences_df_final.drop_duplicates(subset='Full Name', keep='first', inplace=True)

# Create a mapping of original column names to new column names (course ID as integer)
mapping = {}
# course_full_codes = []

for column_name in preference_columns:
    # Extract the course ID number from the column name
    # course_id = column_name.split(' || ')[0].split(' - ')[3] + " || " + column_name.split(' || ')[1] + ' || ' +  column_name.split(' || ')[2].split(' - ')[0]
    course_id = column_name.split(' || ')[0].split(' - ')[3] + " || "  + column_name.split(' || ')[0].split(' - ')[4]+ " || " + column_name.split(' || ')[1] + ' || ' +  column_name.split(' || ')[2].split(' - ')[0]
    
    # Extract full course ID for matching later
    # course_full_codes.append(column_name.split(' || ')[0].split(' - ')[3] + " || "  + column_name.split(' || ')[0].split(' - ')[4]+ " || " + column_name.split(' || ')[1] + ' || ' +  column_name.split(' || ')[2].split(' - ')[0])
    
    # Map the original column name to the course ID
    mapping[column_name] = course_id

# Remove duplicates
# course_full_codes = list(set(course_full_codes))

# Rename the columns using the mapping
preferences_df_final.rename(columns=mapping, inplace=True)

# Drop columns with list of courses (redundant) [30, 81, and 82]
preferences_df_final.drop(columns=preferences_df_final.iloc[:,[30, 81, 82]], inplace=True)

# OUTPUT #2
# Filter the DataFrame for rows where "Do you intend to continue your collaboration with Nova SBE next semester as Teaching Assistant?" 
# (column 20) is not equal to "No"
ta_exits = preferences_df_final[preferences_df_final.iloc[:, 19] == "No"].iloc[:, [17, 18, 20]]
ta_exits = ta_exits.rename(columns={ta_exits.columns[-1]: "Comments"}).sort_values("Full Name")

# ta_exits.to_excel("ta_exits_v1.xlsx", index=False)
ta_exits_list = ta_exits.TA.unique()

preferences_df_final = preferences_df_final[preferences_df_final.iloc[:, 19] != "No"]

In [ ]:
column_17 = column_df[column_df['Column Name'] == "Full Name"].iloc[0]["Column Number"]
column_18 = column_df[column_df['Column Name'] == "TA"].iloc[0]["Column Number"]

continue_str = "Do you intend to continue your collaboration with Nova SBE next semester"
continue_just_str = "Please write here a short justification on why you do not intend to continue"
bs_or_ms_str = "Do you prefer to be assigned to Bachelor’s or Master's courses?"
load_availability_str = "What is your availability in terms of workload and contract percentage for the next semester?"
ms_student_str = "In the upcoming semester, are you going to be a Nova SBE student?"
phd_restrictions_str = "Being a PhD student, do you have any constraint in the number of teaching hours or contract percentage"
new_workload_str = "What is your availability in terms of workload and contract percentage for the next semester"

column_19 = column_df[column_df['Column Name'].str.startswith(continue_str)].iloc[0]["Column Number"]
column_20 = column_df[column_df['Column Name'].str.startswith(continue_just_str)].iloc[0]["Column Number"]
column_21 = column_df[column_df['Column Name'].str.startswith(ms_student_str)].iloc[0]["Column Number"]
column_22 = column_df[column_df['Column Name'].str.startswith(bs_or_ms_str)].iloc[0]["Column Number"]
column_23 = column_df[column_df['Column Name'].str.startswith(phd_restrictions_str)].iloc[0]["Column Number"]
column_27 = column_df[column_df['Column Name'].str.startswith(load_availability_str)].iloc[0]["Column Number"]
column_28 = column_27 + 1
column_29 = column_28 + 1 # Be careful! This assumes there are TWO text boxes for available workload percentage

bs_str = "Please choose below your teaching preferences for Bachelor Courses."
column_30 = column_df[column_df['Column Name'].str.startswith(bs_str)].iloc[0]["Column Number"]
column_31 = column_30 + 1 # BE careful! This assumes there is ONE open text columns for bachelors preferences

ms_str = "Please choose below your teaching preferences for Masters Courses (grading)."
column_81 = column_df[column_df['Column Name'].str.startswith(ms_str)].iloc[0]["Column Number"]
column_82 = column_81 + 1 # BE careful! This assumes there are TWO open text columns for master preferences
column_83 = column_81 + 2 # BE careful! This assumes there are TWO open text columns for master preferences

print(column_17, column_18, column_19, column_20, column_21, column_22, column_23, column_27, column_28, column_29, column_30, column_31, column_81, column_82, column_83)

In [ ]:
column_df.iloc[column_28]

In [ ]:
print(f"Number of TAs who are leaving: {len(ta_exits_list)}")

In [ ]:
# Check that can use the email as key/ ID
preferences_df_final["TA"].isnull().sum()

In [ ]:
# Check there are no duplicates names
preferences_df_final[preferences_df_final["Full Name"].duplicated()]["Response ID"].count()

In [ ]:
# Check there are no duplicates answers
preferences_df_final[preferences_df_final["TA"].duplicated()]["Response ID"].count()

In [ ]:
responded_number = preferences_df_final.shape[0] 
print(f"Number of TAs willing to continue who responded: {responded_number}")
display(preferences_df_final[["TA", "Full Name"]].head())

In [ ]:
# OUTPUT #N
preferences_df_final[~preferences_df_final.iloc[:,-1].isna()].iloc[:, [17, 18, -1]]

In [ ]:
# OUTPUT #N
preferences_df_final[~preferences_df_final["TA"].isin(contract_emails)][["TA", "Full Name"]]

In [ ]:
# Get the course columns
course_columns = preferences_df_final.columns[30:-1]

# Create a new DataFrame for the adapted format
adapted_df = pd.DataFrame(columns=["TA", "course", "preference", "preference_type"])

# Define the translation mapping for column 22 values
translation_mapping = {
    "Masters' Courses": 2,
    "Bachelors' Courses": 0,
    "Indifferent": 1,
    pd.NaT: 1  # Assuming NaN values should also be considered "Indifferent"
}

# Iterate over the course columns
for course in course_columns:
    # Check if the course has already been processed
    if course in adapted_df["course"].unique():
        continue

    # Get the duplicate columns for the current course
    duplicate_columns = [col for col in course_columns if col != course and col.endswith(course)]

    # Combine the duplicate columns into a single column
    combined_column = preferences_df_final[[course] + duplicate_columns].ffill(axis=1).iloc[:, -1]

    # Filter the DataFrame for non-null values in the combined column
    non_null_mask = combined_column.notnull()
    non_null_df = preferences_df_final[non_null_mask]

    # Get the teacher names and their corresponding preference rankings for the current course
    teacher_names = non_null_df["TA"]
    preference_rankings = combined_column[non_null_mask]

    # Get the corresponding preference types based on the translation mapping
    preference_types = non_null_df.iloc[:, 22].map(translation_mapping)

    # Create a DataFrame for the current course, preference rankings, and preference types
    course_df = pd.DataFrame({"TA": teacher_names, "course": [course] * len(teacher_names),
                              "preference": preference_rankings, "preference_type": preference_types})

    # Concatenate course_df with adapted_df
    adapted_df = pd.concat([adapted_df, course_df], ignore_index=True)
    
    # Create the 'masters_course' column based on the condition
    adapted_df['masters_course'] = adapted_df['course'].apply(lambda x: 0 if x.split(' ')[0].startswith('1') else 1)

    # Convert "preference" column to integers
    adapted_df['preference'] = adapted_df['preference'].astype(np.int8)

    # Remove preferences above 5
    adapted_df = adapted_df[adapted_df['preference']<=5]

In [ ]:
# Check there are no preferences above 5 and below 1
print(adapted_df['preference'].min())
print(adapted_df['preference'].max())

In [ ]:
completed_preferences = adapted_df["TA"].unique()
completed_preferences_number = len(completed_preferences)
print(f"Number of TAs with completed preferences: {completed_preferences_number}")
# display(completed_preferences)

In [ ]:
# OUTPUT #6
# ADDED! removed :-1
# adapted_df.iloc[:,:-1].head()
adapted_df.head()

In [ ]:
adapted_df[adapted_df["TA"]=="bernardo.costa@novasbe.pt"]

## B. New contract (preferences)

In [ ]:
contact_list = contract[(~contract.TA.isin(completed_preferences)) & (~contract.TA.isin(ta_exits_list))]
print(contact_list.shape[0])
display(contact_list)
contact_list.to_excel("ta_contact_list_v1.xlsx", index=False)

In [ ]:
def round_to_closest(value):
    if pd.isnull(value):
        return np.nan
    else:
        # rounded_value = round(value * 8) / 8  # Round to the nearest multiple of 0.125
        capped_value = min(value, 0.5)  # Cap the value at 0.5
        capped_value = max(value, 0.1)  # Cap the value at 0.1
        return capped_value

# Clean the "load_requested" column
def clean_percentage(value):
    if pd.isnull(value):
        return value
    elif isinstance(value, str):
        # Check if the value contains only text characters
        if value.isalpha():
            return np.nan

        # Extract numeric values from string
        numeric_value = ''.join(filter(str.isdigit, value))

        if numeric_value == '':
            return np.nan

        if numeric_value == '100':
            return 100

        if len(numeric_value) >= 2:
            integer_part = numeric_value[:2]
            decimal_part = numeric_value[2:]
            return float(integer_part + '.' + decimal_part)

        return np.nan

    elif isinstance(value, (int, float)):
        return float(value) / 100

    return value

def decrease_contract_level(value):
    return value - 0.125


mapping = {
    "I want to increase the contract percentage/workload in the next semester (please specify the desired contract percentage level)": 1,
    "I want to keep the same contract percentage/workload as this semester": 0,
    "I want to reduce the contract percentage/workload in the next semester (please specify the desired contract percentage level)": -1,
    pd.NaT: 0
}

mapping_21 = {
    "Yes, I am a PhD student": 0,
    "Yes, I will be a Masters student but not doing any courses, only the Work Project": 0,
    "Yes, I will be a Masters student and I will be doing at least one more course": 1,
    "No": 0,
    pd.NaT: 0
}

mapping_23 = {
    "Yes, I have some other constraints that limit my teaching hours/workload (please specify the reason and the limit)": 1,
    "Yes, I have a FCT scholarship that limits my weekly teaching hours to 4h per week": 1,
    "No": 0,
    pd.NaT: 0
}

################################################################################################################################################

mask = preferences_df_final.iloc[:, 27].notna()
new_contract = preferences_df_final[mask].iloc[:, [18, 21, 23, 27, 28, 29]]

new_contract.columns = ['TA', 'master_student', 'PhD_restrictions', 'change_load', 'new_contract_decreased_load', 'new_contract_increased_load']
new_contract['change_load'] = new_contract['change_load'].map(mapping)
new_contract['master_student'] = new_contract['master_student'].map(mapping_21).fillna(0).astype(int)
new_contract['PhD_restrictions'] = new_contract['PhD_restrictions'].map(mapping_23).fillna(0).astype(int)

# Convert TA column to lowercase
new_contract['TA'] = new_contract['TA'].str.lower()

new_contract['new_contract_decreased_load'] = new_contract['new_contract_decreased_load'].apply(clean_percentage) / 100
new_contract['new_contract_increased_load'] = new_contract['new_contract_increased_load'].apply(clean_percentage) / 100

# Merge "new_contract_decreased_load" and "new_contract_increased_load" into "load_requested"
new_contract['load_requested'] = new_contract[['new_contract_decreased_load', 'new_contract_increased_load']].mean(axis=1)
new_contract['load_requested'] = new_contract['load_requested'].apply(round_to_closest)

# Drop "new_contract_decreased_load" and "new_contract_increased_load" columns
new_contract.drop(columns=['new_contract_decreased_load', 'new_contract_increased_load'], inplace=True)

In [ ]:
# OUTPUT #3
new_contract[new_contract.change_load !=0].sort_values(by="change_load",ascending=False)

In [ ]:
# Check new_contract_load is formatted correctly
# new_contract.load_requested.value_counts()

In [ ]:
number_TAS = len(new_contract[new_contract.change_load.isin([1,-1])])
print(f"Number of TAs who want to change their contract: {number_TAS}")

In [ ]:
all_contracts = contract.merge(new_contract, how="left", on="TA")

# Filter rows where change_load is not equal to 0
filtered_contracts = all_contracts[all_contracts['change_load'] != 0].copy()

# Decrease contract to load_requested for rows where change_load is -1
filtered_contracts.loc[filtered_contracts['change_load'] == -1, 'new_contract'] = filtered_contracts['load_requested']
# ADDED! Caps "CONTRACT"
filtered_contracts.loc[(filtered_contracts['change_load'] == -1) & (filtered_contracts['load_requested'].isnull()), 'new_contract'] = filtered_contracts.apply(lambda row: decrease_contract_level(row['CONTRACT']), axis=1)

# # Increase contract for no restrictions
# filtered_contracts.loc[(filtered_contracts['change_load'] == 1) & (filtered_contracts['restrictions'] == 0), 'new_contract'] = filtered_contracts['load_requested']
# filtered_contracts.loc[(filtered_contracts['change_load'] == 1) & (filtered_contracts['restrictions'] == 0) & (filtered_contracts['load_requested'].isnull()), 'new_contract'] = np.minimum(filtered_contracts['contract'] * 2, 0.5)

# Apply conditions for updating new_contract based on change_load and restrictions
# filtered_contracts.loc[(filtered_contracts['change_load'] == 1) & (filtered_contracts['restrictions'] == 1), 'new_contract'] = filtered_contracts['contract']
# filtered_contracts.loc[(filtered_contracts['change_load'] == 1) & (filtered_contracts['restrictions'] == 2), 'new_contract'] = np.maximum(filtered_contracts['contract'], 0.25)

# Fill NaN values with the original contract value
# ADDED! Caps "CONTRACT"
filtered_contracts['new_contract'].fillna(filtered_contracts['CONTRACT'], inplace=True)

# Create a new column "new_contract" in the original DataFrame with NaN values
all_contracts['new_contract'] = np.nan

# Update the "new_contract" column in the original DataFrame with the filtered values
all_contracts.update(filtered_contracts[['new_contract']])
# ADDED! Caps "CONTRACT"
all_contracts['new_contract'].fillna(all_contracts['CONTRACT'], inplace=True)

# Round the "new_contract" values to the closest contract level (0.125, 0.25, 0.375, 0.5)
# all_contracts['new_contract'] = all_contracts['new_contract'].apply(round_to_closest)

# Drop emails which currently do not have a contract (ex. pedro.brinca)
# ADDED! Caps "CONTRACT"
all_contracts = all_contracts[all_contracts.CONTRACT.notna()]

In [ ]:
# Check there are no TAs wihout current contract
all_contracts[all_contracts.CONTRACT.isnull()].TA.to_list()

In [ ]:
all_contracts = all_contracts[["TA", "new_contract", "master_student"]]

## C. Merge SUPPLY and DEMAND

In [ ]:
ta_preferences = adapted_df.merge(all_contracts, how="left", on="TA", indicator=True)
ta_preferences.head()

In [ ]:
# Check for non-matching TAs
non_matching_values = ta_preferences[ta_preferences['_merge'] != 'both']
ta_preferences.drop(columns=["_merge"], inplace=True)
display(non_matching_values)

In [ ]:
market = ta_preferences.merge(course_demand, how="left", on="course", indicator=True)

# market['number_classes'] = pd.to_numeric(market['number_classes'], errors='coerce').astype(pd.Int64Dtype())
# market['number_students'] = pd.to_numeric(market['number_students'], errors='coerce').astype(pd.Int64Dtype())

In [ ]:
non_matching_values = market[market['_merge'] != 'both']
market.drop(columns=["_merge"], inplace=True)

non_matching_courses = non_matching_values[["course"]].drop_duplicates()
non_matching_courses = non_matching_courses.copy()
non_matching_courses[["course_code", "course_name", "period", "language"]] = non_matching_courses["course"].str.split(" \|\| ", expand=True)

course_demand_extended = course_demand.copy()
course_demand_extended[["course_code", "course_name", "period", "language"]] = course_demand["course"].str.split(" \|\| ", expand=True)
course_demand_extended = course_demand_extended[["course", "course_code", "course_name", "period", "language"]]

In [ ]:
# Initialize an empty DataFrame to store the concatenated results
concatenated_matches = pd.DataFrame()

# Merge on 'course_code', 'period', and 'language'
merged_courses = pd.merge(non_matching_courses, course_demand_extended, on=["course_code", "period", "language"], how="left", suffixes=("", "_new"))
still_unmatched = merged_courses[merged_courses["course_new"].isna()][["course", "course_name", "course_code", "period", "language"]]
concatenated_matches = pd.concat([concatenated_matches, merged_courses[~merged_courses["course_new"].isna()][["course", "course_new"]]])

# Merge on 'course_name', 'period', and 'language'
merged_courses = pd.merge(still_unmatched, course_demand_extended, on=["course_name", "period", "language"], how="left", suffixes=("", "_new"))
still_unmatched = merged_courses[merged_courses["course_new"].isna()][["course", "course_name", "course_code", "period", "language"]]
concatenated_matches = pd.concat([concatenated_matches, merged_courses[~merged_courses["course_new"].isna()][["course", "course_new"]]])

# Merge on 'course_code' and 'period'
merged_courses = pd.merge(still_unmatched, course_demand_extended, on=["course_code", "period"], how="left", suffixes=("", "_new"))
still_unmatched = merged_courses[merged_courses["course_new"].isna()][["course", "course_name", "course_code", "period", "language"]]
concatenated_matches = pd.concat([concatenated_matches, merged_courses[~merged_courses["course_new"].isna()][["course", "course_new"]]])

# # [DANGER! Might include courses from different semester] Merge on 'course_code' and 'language' 
# merged_courses = pd.merge(still_unmatched, course_demand_extended, on=["course_code", "language"], how="left", suffixes=("", "_new"))
# still_unmatched = merged_courses[merged_courses["course_new"].isna()][["course", "course_name", "course_code", "period", "language"]]
# concatenated_matches = pd.concat([concatenated_matches, merged_courses[~merged_courses["course_new"].isna()][["course", "course_new"]]])

# Merge concatenated_matches on the market DataFrame to add the "course_new" column
market = pd.merge(market, concatenated_matches[["course", "course_new"]], on=["course"], how="left")
market["course_new"].fillna(market["course"], inplace=True)
market.rename(columns={"course": "course_old"}, inplace=True)
market.drop(columns=["course_old"], inplace=True)
market.rename(columns={"course_new": "course"}, inplace=True)

# Merge market and course_demand on "course" column
merged_market = pd.merge(market, course_demand[["course", "number_classes", "number_students"]], on="course", how="left", suffixes=("", "_demand"))

# Fill NaN values in number_classes and number_students columns
merged_market["number_classes"].fillna(merged_market["number_classes_demand"], inplace=True)
merged_market["number_students"].fillna(merged_market["number_students_demand"], inplace=True)

# Drop the unnecessary columns
merged_market.drop(columns=["number_classes_demand", "number_students_demand"], inplace=True)

In [ ]:
# OUTPUT #4
# Display the updated merged_market DataFrame
no_matches_final = merged_market[(merged_market.number_classes.isna()) | (merged_market.number_students.isna())][["course"]]
no_matches_final = no_matches_final.drop_duplicates()
display(no_matches_final)

# Drop these courses
merged_market.dropna(subset=["number_classes", "number_students"], inplace=True)

## D. Compute capacity

In [ ]:
# Create the "semester" column based on the condition
merged_market['semester'] = merged_market['course'].apply(lambda x: 1 if x.split(' || ')[2].startswith('S') else 0)
# merged_market['ms_capacity'] = merged_market['new_contract'] * 36

display(merged_market[(merged_market['semester'].isnull() | merged_market['masters_course'].isnull())])

# Define a function to apply the conditions
def calculate_weight(row):
    if pd.isnull(row['semester']) or pd.isnull(row['masters_course']):
        return np.nan
    elif row['semester'] == 1 and row['masters_course'] == 1:
        return ((row['number_students'] * 2.33) / 16) / 36
    elif row['semester'] == 0 and row['masters_course'] == 1:
        return ((row['number_students'] * 1.25) / 16) / 36
    else:
        return np.nan

# Apply the function to create the 'ms_weight' column
merged_market['weight'] = merged_market.apply(calculate_weight, axis=1)

# Set 'ms_capacity' to NaN when 'masters_course' is 0
# merged_market.loc[merged_market['masters_course'] == 0, 'ms_capacity'] = np.nan
# merged_market.loc[merged_market['masters_course'] == 0, 'ms_weight'] = np.nan

In [ ]:
merged_market[merged_market.TA=="bernardo.costa@novasbe.pt"]

In [ ]:
merged_market.masters_course.value_counts(dropna=False)

### Get file course list to manually input the weights

In [ ]:
course_demand_extended['masters_course'] = course_demand_extended['course'].apply(lambda x: 0 if x.split(' ')[0].startswith('1') else 1)
course_demand_extended_bs = course_demand_extended[course_demand_extended.masters_course==0]
course_demand_extended_bs = course_demand_extended_bs.drop(columns=["masters_course"])
course_demand_extended_bs["weight"] = ""
course_demand_extended_bs.to_excel("bs_courses_weights_EMPTY.xlsx", index=False)

### Read the manually inputed weights

In [ ]:
bs_weights_df = pd.read_excel(bs_weights)[["course", "weight"]]
bs_weights_df["weight"] = bs_weights_df["weight"] * 0.125
bs_weights_df.weight.value_counts(dropna=False)

In [ ]:
final_market = pd.merge(merged_market, bs_weights_df, on=["course"], how="left", suffixes=("", "_bs"), indicator=True)
final_market.head()

In [ ]:
# Check there are no BS courses without weight
final_market[(final_market["_merge"] != "both") & (final_market.masters_course==0)]

In [ ]:
final_market.rename(columns={"new_contract": "capacity"}, inplace=True) # "weight": "bs_weight", 
final_market.drop(columns="_merge", inplace=True)

# ADDED! multiply by number of classes
final_market["weight"] = final_market["weight"].fillna(final_market["weight_bs"] * final_market["number_classes"])
final_market.drop(columns=["weight_bs"], inplace=True)

# final_market.loc[final_market['masters_course'] == 1, 'bs_capacity'] = np.nan
# final_market.loc[final_market['masters_course'] == 1, 'bs_weight'] = np.nan

In [ ]:
tas = final_market.TA.unique()

# Find elements missing in array1 compared to array2
missing_elements = np.setdiff1d(completed_preferences, tas)

# OUTPUT #N: TAs affected by unmatched courses
print(missing_elements)

### Examples

In [ ]:
# final_market[final_market.TA=="bernardo.costa@novasbe.pt"].sort_values(by=['preference_type', 'preference'],ascending=[False,True])
final_market[final_market.TA=="fabian.wassmann@novasbe.pt"].sort_values(by=['preference_type', 'preference'],ascending=[False,True])

In [ ]:
final_market[final_market.course=='2866 || Work Project* || S1 || EN'].sort_values(by=['preference_type', 'preference'],ascending=[False,True]) # '1117 || Introdução à Microeconomia || S1 || EN'
# final_market[final_market.course=="2597 || Advanced Data Analysis || S1 || EN"].sort_values(by=['preference_type', 'preference'],ascending=[False,True])


In [ ]:
# final_market[final_market.course=='1117 || Introdução à Microeconomia || S1 || EN'].sort_values(by=['preference', 'preference_type'],ascending=[True,True])
# final_market[final_market.course=='1118 || Introdução à Macroeconomia || S1 || PT'].sort_values(by=['preference_type', 'preference'],ascending=[True,True])
final_market[final_market.course== '1318 || Programação || S1 || EN'].sort_values(by=['preference_type', 'preference'],ascending=[True,True])

# Matching algorithm

In [ ]:
translation_mapping

In [ ]:
ta_dict = final_market[['TA','capacity']].drop_duplicates()
ta_dict = dict(zip(ta_dict['TA'], ta_dict['capacity']))

In [ ]:
# ADDED 'preference_type' == 2
ms_courses = final_market[(final_market['masters_course'] == 1) & (final_market['master_student'] == 0) & ((final_market['preference_type'] == 2) | (final_market['preference_type'] == 1)) & (final_market['preference'] == 1)]

ms_courses_dict = ms_courses[['course','weight']].drop_duplicates()
ms_courses_dict = dict(zip(ms_courses_dict['course'], ms_courses_dict['weight']))

# ADDED 'preference_type' == 2
bs_courses = final_market[(final_market['masters_course'] == 0) & ((final_market['preference_type'] == 0) | (final_market['preference_type'] == 1)) & (final_market['preference'] == 1)]

bs_courses_dict = bs_courses[['course','weight']].drop_duplicates()
bs_courses_dict = dict(zip(bs_courses['course'], bs_courses['weight']))

In [ ]:
ms_final_preferences = ms_courses[["TA", "preference_type", "preference", "course", "semester"]]
ms_final_preferences = ms_final_preferences.sort_values(by=["course", "preference_type", "preference"], ascending=[True, False, True])

bs_final_preferences = bs_courses[["TA", "preference_type", "preference", "course", "semester"]]
bs_final_preferences = bs_final_preferences.sort_values(by=["course", "preference_type", "preference"], ascending=[True, True, True])

In [ ]:
print(len(ms_final_preferences.course.unique()))
print(len(bs_final_preferences.course.unique()))

In [ ]:
# # Count the number of observations for each course and preference
# course_preference_counts = ms_final_preferences.groupby(['course', 'preference']).size().reset_index(name='count')
# # Filter out the courses with more than one observation for preference == 1
# filtered_courses = course_preference_counts.loc[(course_preference_counts['preference'] == 1) & (course_preference_counts['count'] == 1), 'course']
# # Filter the original DataFrame based on the filtered courses
# ms_final_preferences_filtered = ms_final_preferences[ms_final_preferences['course'].isin(filtered_courses)]

# # Print the filtered DataFrame
# print(len(ms_final_preferences_filtered.course.unique()))

In [ ]:
# # Count the number of observations for each course and preference
# course_preference_counts = bs_final_preferences.groupby(['course', 'preference']).size().reset_index(name='count')
# # Filter out the courses with more than one observation for preference == 1
# filtered_courses = course_preference_counts.loc[(course_preference_counts['preference'] == 1) & (course_preference_counts['count'] == 1), 'course']
# # Filter the original DataFrame based on the filtered courses
# bs_final_preferences_filtered = bs_final_preferences[bs_final_preferences['course'].isin(filtered_courses)]

# # Print the filtered DataFrame
# print(len(bs_final_preferences_filtered.course.unique()))

In [ ]:
ta_allocations = []

In [ ]:
for _, row in bs_final_preferences.iterrows():
    ta = row['TA']
    course = row['course']
    ta_capacity = ta_dict[ta]
    course_weight = bs_courses_dict[course]
    # Check if course can be allocated
    if course_weight > 0:
        # Check if TA still has capacity
        if  ta_capacity > 0:
            allocated_weight = min(course_weight, ta_capacity)
            # Allocate course to TA
            ta_allocations.append((ta, course, allocated_weight))
            ta_dict[ta] -= allocated_weight
            bs_courses_dict[course] -= allocated_weight
        else:
            try:
                bs_final_preferences = bs_final_preferences[bs_final_preferences['ta'] != ta]
            except:
                continue
    else:
        bs_final_preferences = bs_final_preferences[bs_final_preferences['course'] != course]

In [ ]:
for _, row in ms_final_preferences.iterrows():
    ta = row['TA']
    course = row['course']
    ta_capacity = ta_dict[ta]
    course_weight = ms_courses_dict[course]
    # Check if course can be allocated
    if course_weight > 0:
        # Check if TA still has capacity
        if  ta_capacity > 0:
            allocated_weight = min(course_weight, ta_capacity)
            # Allocate course to TA
            ta_allocations.append((ta, course, allocated_weight))
            ta_dict[ta] -= allocated_weight
            ms_courses_dict[course] -= allocated_weight
        else:
            try:
                ms_final_preferences = ms_final_preferences[ms_final_preferences['ta'] != ta]
            except:
                continue
    else:
        ms_final_preferences = ms_final_preferences[ms_final_preferences['course'] != course]

### Results

#### Get all courses and the respective capacities

In [ ]:
full_courses.head()

In [ ]:
full_courses.CYCLE.value_counts(dropna=False)

In [ ]:
full_courses[full_courses.course=="127895 || Space for Business || T2 || EN"]

In [ ]:
bs_weights_df[bs_weights_df.course=="1114 || Economia Global II || S1 || EN"]

In [ ]:
# Merging DataFrames
full_course_weights = full_courses.merge(bs_weights_df, on="course", how="left")
full_course_weights.rename(columns={"course": "COURSE"}, inplace=True)

# Condition: If "CYCLE" == "BSC"
mask_bs = full_course_weights["CYCLE"] == "BSC"
full_course_weights.loc[mask_bs, "INITIAL NEEDS"] = full_course_weights.loc[mask_bs, "CLASS"] * full_course_weights.loc[mask_bs, "weight"]

# Condition: If "CYCLE" == "MST"
mask_ms = full_course_weights["CYCLE"] == "MST" 
def calculate_weight(row):
    if pd.isnull(row['TERM']) or pd.isnull(row['CYCLE']):
        return np.nan
    elif row['TERM'].startswith('S') and row['CYCLE'] == 'MST':
        return ((row['SLOTS'] * 2.33) / 16 ) / 36
    elif row['TERM'].startswith('T') and row['CYCLE'] == 'MST':
        return ((row['SLOTS'] * 1.25) / 16) / 36
    else:
        return np.nan

full_course_weights.loc[mask_ms, "INITIAL NEEDS"] = full_course_weights.loc[mask_ms].apply(calculate_weight, axis=1)
full_course_weights.drop(columns="weight", inplace=True)

In [ ]:
len(ta_allocations)

In [ ]:
final_market[final_market.masters_course==1].head()

In [ ]:
# # Get unique courses from the final_market dataframe
# all_courses = full_course_weights['COURSE'].unique()

# # Create a dataframe for the courses and their needs
# course_needs = pd.DataFrame({
#     "COURSE": all_courses,
#     "INITIAL NEEDS": [final_market[final_market['course'] == course]['weight'].values[0] for course in all_courses],
#     "NEEDS": [ms_courses_dict.get(course, bs_courses_dict.get(course, final_market[final_market['course'] == course]['weight'].values[0])) for course in all_courses],
#     "CYCLE": ["MS" if final_market[final_market['course'] == course]['masters_course'].values[0] == 1 else "BS" for course in all_courses]
# })

# # Multiply NEEDS and INITIAL NEEDS by 36 for CYCLE == MS
# course_needs.loc[course_needs["CYCLE"] == "MS", ["NEEDS", "INITIAL NEEDS"]] *= 36

# # Add the MATCH column based on the conditionsa
# course_needs.loc[course_needs["CYCLE"] == "ME", "MATCH"] = "NO"
# course_needs.loc[course_needs["INITIAL NEEDS"] == course_needs["NEEDS"], "MATCH"] = "NO"
# course_needs.loc[(course_needs["INITIAL NEEDS"] != course_needs["NEEDS"]) & (course_needs["NEEDS"] > 0), "MATCH"] = "PARTIAL"
# course_needs.loc[(course_needs["INITIAL NEEDS"] != course_needs["NEEDS"]) & (course_needs["NEEDS"] == 0), "MATCH"] = "MATCHED"

In [ ]:
# Get unique courses from the full_course_weights dataframe
all_courses = full_course_weights['COURSE'].unique()

# Create a dataframe for the courses and their needs
course_needs = pd.DataFrame({
    "CYCLE": full_course_weights.loc[full_course_weights['COURSE'].isin(all_courses), 'CYCLE'],
    "COURSE": all_courses,
    "TERM": [full_course_weights[full_course_weights['COURSE'] == course]['TERM'].values[0] for course in all_courses],
    "CLASSES": [full_course_weights[full_course_weights['COURSE'] == course]['CLASS'].values[0] for course in all_courses],
    "SLOTS": [full_course_weights[full_course_weights['COURSE'] == course]['SLOTS'].values[0] for course in all_courses],
    "INITIAL NEEDS": [full_course_weights[full_course_weights['COURSE'] == course]['INITIAL NEEDS'].values[0] for course in all_courses],
    "NEEDS": [ms_courses_dict.get(course, bs_courses_dict.get(course, full_course_weights[full_course_weights['COURSE'] == course]['INITIAL NEEDS'].values[0])) for course in all_courses]
    
})

# Multiply NEEDS and INITIAL NEEDS by 36 for CYCLE == MS
course_needs.loc[course_needs["CYCLE"] == "MST", ["NEEDS", "INITIAL NEEDS"]] *= 36

# Add the MATCH column based on the conditionsa
course_needs.loc[course_needs["CYCLE"] == "ME", "MATCH"] = "NO"
course_needs.loc[course_needs["INITIAL NEEDS"] == course_needs["NEEDS"], "MATCH"] = "NO"
course_needs.loc[(course_needs["INITIAL NEEDS"] != course_needs["NEEDS"]) & (course_needs["NEEDS"] > 0), "MATCH"] = "PARTIAL"
course_needs.loc[(course_needs["INITIAL NEEDS"] != course_needs["NEEDS"]) & (course_needs["NEEDS"] == 0), "MATCH"] = "MATCHED"

In [ ]:
print(full_course_weights.shape)
full_course_weights.head()

In [ ]:
print(course_needs.shape)
display(course_needs.head())
course_needs.to_excel("course_needs.xlsx", index=False)

In [ ]:
course_needs.CYCLE.value_counts(dropna=False)

In [ ]:
# Create a dataframe for the TA allocations
ta_allocations_df = pd.DataFrame(ta_allocations, columns=["TA", "COURSE", "LOAD"])
ta_allocations_df["CYCLE"] = ta_allocations_df["COURSE"].apply(lambda x: "MST" if x in ms_courses_dict else "BSC")

In [ ]:
print(ta_allocations_df.shape)
new_order = ['CYCLE', 'COURSE', 'TA', 'LOAD']
ta_allocations_df = ta_allocations_df[new_order]
display(ta_allocations_df.head())
ta_allocations_df.to_excel("ta_allocations_auto.xlsx", index=False)

In [ ]:
# ms_courses_dict_filtered = {k: v for k, v in ms_courses_dict.items() if v == 0}
# len(ms_courses_dict_filtered)

In [ ]:
# ms_courses_dict_filtered = {k: v for k, v in ms_ta_dict.items() if v < 5}
# len(ms_courses_dict_filtered)
# ms_courses_dict_filtered

### Version combined

In [ ]:
# courses_dict = final_market[['course','weight']].drop_duplicates()
# courses_dict = dict(zip(courses_dict['course'], courses_dict['weight']))

In [ ]:
# course_preference_counts = final_preferences.groupby(['course', 'preference']).size().reset_index(name='count')

# # Filter out the courses with more than one observation for preference == 1
# filtered_courses = course_preference_counts.loc[(course_preference_counts['preference'] == 1) & (course_preference_counts['count'] == 1), 'course']
# final_preferences_filtered = final_preferences[final_preferences['course'].isin(filtered_courses)]

# # Print the filtered DataFrame
# print(len(final_preferences_filtered.course.unique()))

In [ ]:
# final_preferences = final_market[["TA", "preference_type", "preference", "course", "semester", "master_student", "masters_course"]]
# final_preferences = final_preferences.sort_values(by=["course", "preference_type", "preference"], ascending=[True, False, True])
# final_preferences.head()

In [ ]:
# len(final_preferences.course.unique())

In [ ]:
# ta_allocations = []

# for _, row in final_preferences_filtered.iterrows():
#     ta = row['TA']
#     pref_type = row['preference_type']
#     course = row['course']
#     student = row['master_student']
#     ms_course = row['masters_course']

#     ta_capacity = ta_dict[ta]
#     course_weight = courses_dict[course]

#     # MS courses
#     if ms_course == 1 and student == 0:
#         # Give priority to TAs who want MS courses
#         if pref_type == 1 or pref_type == 2:
#             # Check if course can be allocated
#             if course_weight > 0:
#                 # Check if TA still has capacity
#                 if  ta_capacity > 0:
#                     allocated_weight = min(course_weight, ta_capacity)
#                     # Allocate course to TA
#                     ta_allocations.append((ta, course, allocated_weight))
#                     ta_dict[ta] -= allocated_weight
#                     courses_dict[course] -= allocated_weight
#                 else:
#                     try:
#                         final_preferences_filtered = final_preferences_filtered[final_preferences_filtered['ta'] != ta]
#                     except:
#                         continue
#             else:
#                 final_preferences_filtered = final_preferences_filtered[final_preferences_filtered['course'] != course]
#         else:
#             continue
    
#     elif ms_course == 0:
#         # Give priority to TAs who want BS courses
#         if pref_type == 0:
#             # Check if course can be allocated
#             if course_weight > 0:
#                 # Check if TA still has capacity
#                 if  ta_capacity > 0:
#                     allocated_weight = min(course_weight, ta_capacity)
#                     # Allocate course to TA
#                     ta_allocations.append((ta, course, allocated_weight))
#                     ta_dict[ta] -= allocated_weight
#                     courses_dict[course] -= allocated_weight
#                 else:
#                     try:
#                         final_preferences_filtered = final_preferences_filtered[final_preferences_filtered['ta'] != ta]
#                     except:
#                         continue
#             else:
#                 final_preferences_filtered = final_preferences_filtered[final_preferences_filtered['course'] != course]
#         else:
#             continue
#     else:
#         if course_weight > 0:
#             # Check if TA still has capacity
#             if  ta_capacity > 0:
#                 allocated_weight = min(course_weight, ta_capacity)
#                 # Allocate course to TA
#                 ta_allocations.append((ta, course, allocated_weight))
#                 ta_dict[ta] -= allocated_weight
#                 courses_dict[course] -= allocated_weight
#             else:
#                 try:
#                     final_preferences_filtered = final_preferences_filtered[final_preferences_filtered['ta'] != ta]
#                 except:
#                     continue
#         else:
#             final_preferences_filtered = final_preferences_filtered[final_preferences_filtered['course'] != course]            